In [0]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
#izracunati udio koliko ih je u zraku i na papiru

In [2]:
!pip install ipython-autotime

%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1831 sha256=a08cc467c8522b9ef9d5d7ff8291e3c9e1eef1add829384f02223efda5ee2b14
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [3]:
2+2

4

time: 6.54 ms


In [0]:
proba_data = pd.read_csv('/content/drive/My Drive/u00020s00001_hw00002.svc', delim_whitespace=True, skiprows=[0], header=None)

time: 418 ms


In [0]:
proba_data.head(5)

,0,1,2,3,4,5,6
0,50137,7719,12291055,1,1970,560,14
1,50150,7719,12291063,1,1970,560,43
2,50155,7719,12291070,1,1970,560,67
3,50160,7719,12291078,1,1970,560,81
4,50165,7719,12291085,1,1970,560,99


time: 25.6 ms


Računanje ukupnog vremena, prosječnog pritiska i razlike pritiska. Broj poteza na papiru i u zraku.

In [0]:
proba_data = proba_data.rename(columns={0: "X_pos", 1: "Y_pos", 2: "TimeStamp", 3: "Status", 4: "Azimuth", 5: "Altitude", 6: "Pressure"})
time = np.max(proba_data["TimeStamp"])-np.min(proba_data["TimeStamp"])
proba_data["ShiftStatus"] = proba_data["Status"].shift(-1)
proba_data["ShiftStatus"].loc[np.size(proba_data,0)-1]=1
proba_data["ShiftStatus"] = proba_data["ShiftStatus"] - proba_data["Status"]
press_data = proba_data[proba_data["Pressure"]!=0]
press_mean = press_data["Pressure"].mean()
press_min = press_data["Pressure"].min()
press_max = press_data["Pressure"].max()
stroke_air = np.size(proba_data[proba_data["ShiftStatus"]==-1],0)
stroke_paper = np.size(proba_data[proba_data["ShiftStatus"]==1],0)

time: 101 ms


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Izračunate prosječne brzine za poteze u zraku i na papiru, udio točaka na papiru i u zraku.

In [0]:
proba_data["ShiftX"] = proba_data["X_pos"].shift(-1)
proba_data["ShiftY"] = proba_data["Y_pos"].shift(-1)
proba_data["ShiftTime"] = proba_data["TimeStamp"].shift(-1)

proba_data.at[proba_data.index[-1], 'ShiftTime'] = proba_data.at[proba_data.index[-1], 'TimeStamp']
proba_data.at[proba_data.index[-1], 'ShiftX'] = proba_data.at[proba_data.index[-1], 'X_pos']
proba_data.at[proba_data.index[-1], 'ShiftY'] = proba_data.at[proba_data.index[-1], 'Y_pos']
proba_data["ShiftTime"] = proba_data["ShiftTime"] - proba_data["TimeStamp"]
proba_data["ShiftX"] = proba_data["ShiftX"] - proba_data["X_pos"]
proba_data["ShiftY"] = proba_data["ShiftY"] - proba_data["Y_pos"]
proba_data["Brzina"] = np.sqrt(proba_data["ShiftX"] * proba_data["ShiftX"] + proba_data["ShiftY"] * proba_data["ShiftY"])
#proba_data["Brzina"] = proba_data["Brzina"]/proba_data["ShiftTime"]

time: 22.2 ms


In [0]:
proba_data["Brzina"] = proba_data["Brzina"]/proba_data["ShiftTime"]

time: 3.62 ms


In [0]:
on_paper = proba_data[proba_data["Status"] == 1]
brzina_paper = on_paper["Brzina"].sum()/np.size(on_paper["Brzina"],0)
in_air = proba_data[proba_data["Status"] == 0]
brzina_zrak = in_air["Brzina"].sum()/np.size(in_air["Brzina"],0)

time: 11.8 ms


In [0]:
PerInAir = np.size(in_air,0)/np.size(proba_data,0)
PerOnPaper = np.size(on_paper,0)/np.size(proba_data,0)

time: 4.96 ms


In [0]:
print(PerInAir)
print(PerOnPaper)

0.48117671261057393
0.5188232873894261
time: 1.91 ms


Izračunata vremena provedena na papiru i u zraku.

In [0]:
StatusChangeData = proba_data[proba_data["ShiftStatus"]!=0]
StatusChangeData["TimeDiff"] = StatusChangeData["TimeStamp"].shift(-1)
StatusChangeData.at[StatusChangeData.index[0], 'TimeStamp'] = proba_data.at[proba_data.index[0], 'TimeStamp']
StatusChangeData.at[StatusChangeData.index[-1], 'TimeDiff'] = proba_data.at[proba_data.index[-1], 'TimeStamp']
StatusChangeData["TimeDiff"] = StatusChangeData["TimeDiff"] - StatusChangeData["TimeStamp"]
statusone = StatusChangeData[StatusChangeData["Status"]==1]
summa = statusone['TimeDiff'].sum()
statuszero = StatusChangeData[StatusChangeData["Status"]==0]
summazero = statuszero['TimeDiff'].sum()
#StatusChangeData =StatusChangeData.drop(columns = ["TimeDiff"])
#StatusChangeData do feat stavljeno sve u ovu celiju

time: 118 ms


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


28 prelazaka iz papira u zrak i obratno. 28 poteza u zraku i 29 na papiru?

In [0]:
feat = pd.DataFrame(columns = ["TotalTime" , "TimeOnPaper", "TimeInAir", "AvgPressure", "DiffPressure", "StrokesInAir", "StrokesOnPaper", "PerStrInAir","PerStrOnPaper","VelAir", "VelPaper" ])
feat = feat.append({"TotalTime":time , "TimeOnPaper":summa, "TimeInAir":summazero, "AvgPressure":press_mean, "DiffPressure":press_max - press_min, "StrokesInAir":stroke_air, "StrokesOnPaper":stroke_paper, "VelAir":brzina_paper, "VelPaper":brzina_zrak, "PerStrOnPaper":PerOnPaper, "PerStrInAir":PerInAir}, ignore_index = True)
if proba_data.at[proba_data.index[0], 'TimeStamp']==1:
  feat.at[feat.index[0],"StrokesOnPaper"] = feat.at[feat.index[0],"StrokesOnPaper"] + 1
else:
  feat.at[feat.index[0],"StrokesInAir"] = feat.at[feat.index[0],"StrokesInAir"] + 1

time: 23 ms


In [0]:
feat

,TotalTime,TimeOnPaper,TimeInAir,AvgPressure,DiffPressure,StrokesInAir,StrokesOnPaper,PerStrInAir,PerStrOnPaper,VelAir,VelPaper
0,38359.0,20230.0,18129.0,334.248314,594.0,29.0,28.0,0.481177,0.518823,3.275331,5.283136


time: 26.8 ms


In [0]:
#feat.to_csv(file_name, header=False, mode = 'a') izracunajmo znacajke za taj zadatak i stavljamo na kraj datoteke file_name